# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from datetime import datetime

import regex as re
from bs4 import BeautifulSoup 

# Data Cleaning: 
1. Language tags 
    - Target Tags: 
        - SQL (Target Class: 0)
        - Scala (Target Class: 1)
        - R (Target Class: 2)
        - Julia (Target Class: 3)
        - C++ (Target Class: 4)
        - Python (Target Class: 5)
    - Remove tags that won't be used for modeling (the 6 above) 
2. Body & Title (will I use Title in modeling?) 
    - Clean up body and titles:
        - remove HTML tags & numbers 
3. MISC Cleaning
    - Reset Index
    - CreationDate column --> Datetimeformat

#### To continue with this notebook you will need to have weekly_combined2019.csv file. [link](https://drive.google.com/open?id=1OPgd2J14mfQkCpvhz9O8qFbql0FUpDaf)

In [2]:
df = pd.read_csv('../data/weekly_combined2019.csv')

In [3]:
df.head()

,Id,CreationDate,Title,Body,Tags,ViewCount,AnswerCount,CommentCount
0,53992215,2019-01-01 00:00:04,Using a ScheduledExecutorService to run a task...,<p>I am working on a program that will read da...,<java><executorservice><java-threads>,73,1,3
1,53992219,2019-01-01 00:01:55,How to programmatically change style sheet of ...,<p>I have so many buttons on a dialog and I wa...,<c++><qt><qt5><qtstylesheets><qpushbutton>,775,2,2
2,53992220,2019-01-01 00:02:08,Trying to put website into Maintenance Mode (3...,<p>I'm trying to put my webpage into Maintenan...,<.htaccess>,364,2,13
3,53992221,2019-01-01 00:02:27,Node.js Lambda Async return Undefined,<p>Simple call to ec2 Describing Security grou...,<node.js><lambda>,423,3,3
4,53992223,2019-01-01 00:02:37,Unable to print a class list attribute using i...,<p>I am designing a deck class that has <stron...,<python><python-3.x><list><class><printing>,40,2,0


## Language Tags 

### Clean up tags

In [4]:
def tag_replace_split(tag):
    '''
    function to get rid of characters ('><, <, >') from tag column and replace with either space, or nothing 
    then put that new tag into list to be able to parse more easily for modeling  

    '''
    tag = tag.replace('><', ' ')
    tag = tag.replace('<', '')
    tag = tag.replace('>', '')
    tag = tag.split()
    
    return tag 

In [5]:
# Use "tag_replace_split" function to clean up original tags column  
df['Tags'] = df['Tags'].map(lambda x: tag_replace_split(x))

In [6]:
df.head()

,Id,CreationDate,Title,Body,Tags,ViewCount,AnswerCount,CommentCount
0,53992215,2019-01-01 00:00:04,Using a ScheduledExecutorService to run a task...,<p>I am working on a program that will read da...,"[java, executorservice, java-threads]",73,1,3
1,53992219,2019-01-01 00:01:55,How to programmatically change style sheet of ...,<p>I have so many buttons on a dialog and I wa...,"[c++, qt, qt5, qtstylesheets, qpushbutton]",775,2,2
2,53992220,2019-01-01 00:02:08,Trying to put website into Maintenance Mode (3...,<p>I'm trying to put my webpage into Maintenan...,[.htaccess],364,2,13
3,53992221,2019-01-01 00:02:27,Node.js Lambda Async return Undefined,<p>Simple call to ec2 Describing Security grou...,"[node.js, lambda]",423,3,3
4,53992223,2019-01-01 00:02:37,Unable to print a class list attribute using i...,<p>I am designing a deck class that has <stron...,"[python, python-3.x, list, class, printing]",40,2,0


### Categorize Langauge Tags
- categorizing and drop unnecesary rows with non-target tags

#### Language Tag Sets

In [7]:
# Individual tag sets for each language I'm targeting 
# created target sets

# SQL (target class: 0)
sql_set = {'sql','bigquery-standard-sql','sqlparameters', 'sqlparameter', 'web-sql', 'sqlite3','sqlite', 
          'sql2016', 'sql-server-2016', 'sql-execution-plan','spark-sql','sql-order-by','sql-join',  
          'sql-syntax','sql-statement','sql-query','mysql-query','orderby','sql-server','mysql'} 

# Scala (taget class: 1)
scala_set = {'scala', 'scala-implicits', 'scala-generics'} 

# R (target class: 2)
r_set = {'rstats', 'r', 'r-language', 'r-faq'}

# Julia (target: 3)
julia_set = {'julia-lang', 'julia', 'julialang'}

# C++ (target: 4)
cplus_set = {'c++', 'c++17', 'visual-c++'}

# Python (target: 5)
py_set = {'python', 'python3', 'python-3.7', 'python-3.x', 'pytorch', 'python-list', 'py3', 'ipython-notebook', 
          'python3', 'python-3.5.2' , 'python-2.7', 'python-3', 'python-bigquery', 'pythonic', 'py'}

# includes all language tags 
# union of all sets 
master_set = (sql_set|scala_set|r_set|julia_set|cplus_set|py_set)

# else : -1 

all_sets = [sql_set, scala_set, r_set, julia_set, cplus_set, py_set]

In [8]:
# look through each row of tags and see if there's an intersection with tags and master set of languages 
# has to be a set to use .intersection 
df['target_tags'] = df['Tags'].apply(lambda tags: set(tags).intersection(master_set))

In [9]:
df.head()

,Id,CreationDate,Title,Body,Tags,ViewCount,AnswerCount,CommentCount,target_tags
0,53992215,2019-01-01 00:00:04,Using a ScheduledExecutorService to run a task...,<p>I am working on a program that will read da...,"[java, executorservice, java-threads]",73,1,3,{}
1,53992219,2019-01-01 00:01:55,How to programmatically change style sheet of ...,<p>I have so many buttons on a dialog and I wa...,"[c++, qt, qt5, qtstylesheets, qpushbutton]",775,2,2,{c++}
2,53992220,2019-01-01 00:02:08,Trying to put website into Maintenance Mode (3...,<p>I'm trying to put my webpage into Maintenan...,[.htaccess],364,2,13,{}
3,53992221,2019-01-01 00:02:27,Node.js Lambda Async return Undefined,<p>Simple call to ec2 Describing Security grou...,"[node.js, lambda]",423,3,3,{}
4,53992223,2019-01-01 00:02:37,Unable to print a class list attribute using i...,<p>I am designing a deck class that has <stron...,"[python, python-3.x, list, class, printing]",40,2,0,"{python, python-3.x}"


In [10]:
# Original 
df.shape

(1878570, 9)

In [11]:
# drop rows that have empty sets for target_tags 
df.drop(df[df['target_tags'] == set({})].index, inplace = True)

In [12]:
# New df with only target languages from sets created 
# loss about 75% of data from original
df.shape

(465055, 9)

In [2]:
# difference between old and new / old 
((1878570-465055)/1878570)*100

75.24420170661726

#### Overlapping Target Classes

In [14]:
# use this column to check and see which languages are overlapping b/c can't use .unique on list 
df['overlap_tags'] = df['target_tags'].map(lambda tag: ' '.join(tag))

In [15]:
# overlapping languagues: 
# Fair amount, will need to address this. 
# r & python 
# sql & python 
# r & sql
# c ++ & sql 
# 'python-3.x scala sql-server'
# 'python-3.x python-2.7 c++'
# 'python-3.x python r sql'

df['overlap_tags'].unique()

array(['c++', 'python python-3.x', 'python', 'sql', 'r', 'sqlite',
       'sql-server', 'mysql', 'python-3.x', 'scala', 'mysql sql',
       'mysql python python-3.x', 'sql-server sql', 'python python-2.7',
       'pytorch', 'mysql sql python', 'sqlite python', 'pytorch python',
       'python-2.7', 'c++ visual-c++', 'mysql python', 'sql python',
       'c++17 c++', 'mysql c++', 'c++ python',
       'python python-3.x python-2.7', 'sqlite sql', 'visual-c++',
       'sqlite c++ sql', 'mysql sql-server sql', 'mysql sql-order-by',
       'sql sql-server-2016', 'julia', 'mysql sql sql-order-by',
       'sql-server sql-server-2016', 'sqlite python-3.x',
       'mysql python python-2.7', 'sql-server sql sql-server-2016',
       'sqlite sql-server', 'sql-execution-plan sql-server sql',
       'sqlite python-3.x python', 'sql-server python',
       'sql-order-by sql', 'sql-server python-3.x', 'pytorch python-2.7',
       'c++ sql', 'sql-server-2016', 'sql r', 'pytorch python-3.x python',
      

In [16]:
# If have time will come back and create functions to check for overlapping classes 
def check_2class(tags, class1, class2):
    '''
    make a function that would take a post's tags, and check 
    if the tags are in class1 and class2.
    
    If so, return the rows that satisfy that condition.
    
    (Then you can drop those rows --> do this outside the function)
    '''
    pass

def check_3class(tags, class1, class2, class3):
    pass

def check_4class(tags, class1, class2, class3, class4):
    pass

def check_5class(tags, class1, class2, class3, class4, class5):
    pass

def check_6class(tags, class1, class2, class3, class4, class5, class6):
    pass

In [17]:
df.head()

,Id,CreationDate,Title,Body,Tags,ViewCount,AnswerCount,CommentCount,target_tags,overlap_tags
1,53992219,2019-01-01 00:01:55,How to programmatically change style sheet of ...,<p>I have so many buttons on a dialog and I wa...,"[c++, qt, qt5, qtstylesheets, qpushbutton]",775,2,2,{c++},c++
4,53992223,2019-01-01 00:02:37,Unable to print a class list attribute using i...,<p>I am designing a deck class that has <stron...,"[python, python-3.x, list, class, printing]",40,2,0,"{python, python-3.x}",python python-3.x
8,53992234,2019-01-01 00:05:48,How to rearrange subplots so that one is under...,<p>I am trying to code two plots such that one...,"[python, matplotlib, subplot]",519,1,1,{python},python
10,53992248,2019-01-01 00:09:24,Function always returns 1,<p>I´m trying to write a simple branch predict...,"[c++, function]",150,1,21,{c++},c++
11,53992252,2019-01-01 00:11:20,possible to speed up this query?,<p>I have the following query which takes a li...,"[sql, postgresql]",40,1,0,{sql},sql


#### Create Target Classes

- 0 = SQL  
- 1 = Scala  
- 2 = R 
- 3 = Julia  
- 4 = C++  
- 5 = Python 

In [18]:
def assign_target(tags):
    """
    Function to create target classes for my tags 
    
    *Limitation*: uses first tag to assign the target. 
    If there are multiple langauges in the tags like SQL and Python, 
    will pick SQL since appears first. 
    
    """
    for target_class, target_set in enumerate(all_sets):
        for tag in tags.split():
            if tag in target_set:
                return target_class
            else:
                continue
    return -1

In [19]:
df['target_class'] =  df['overlap_tags'].map(assign_target)
df.head()

,Id,CreationDate,Title,Body,Tags,ViewCount,AnswerCount,CommentCount,target_tags,overlap_tags,target_class
1,53992219,2019-01-01 00:01:55,How to programmatically change style sheet of ...,<p>I have so many buttons on a dialog and I wa...,"[c++, qt, qt5, qtstylesheets, qpushbutton]",775,2,2,{c++},c++,4
4,53992223,2019-01-01 00:02:37,Unable to print a class list attribute using i...,<p>I am designing a deck class that has <stron...,"[python, python-3.x, list, class, printing]",40,2,0,"{python, python-3.x}",python python-3.x,5
8,53992234,2019-01-01 00:05:48,How to rearrange subplots so that one is under...,<p>I am trying to code two plots such that one...,"[python, matplotlib, subplot]",519,1,1,{python},python,5
10,53992248,2019-01-01 00:09:24,Function always returns 1,<p>I´m trying to write a simple branch predict...,"[c++, function]",150,1,21,{c++},c++,4
11,53992252,2019-01-01 00:11:20,possible to speed up this query?,<p>I have the following query which takes a li...,"[sql, postgresql]",40,1,0,{sql},sql,0


In [20]:
# no -1, so there aren't any extra classes 
df['target_class'].unique()

array([4, 5, 0, 2, 1, 3])

#### Target Class Baseline 

In [21]:
df['target_class'].value_counts(normalize = True)

5    0.552298
0    0.211044
2    0.110993
4    0.103900
1    0.019045
3    0.002720
Name: target_class, dtype: float64

## Body Cleanup
    - future iterations of the project, will consider adding titles as well. 

In [22]:
def body_cleanup(raw_body):
    '''
    Function to convert original body to a string of words: 
    - remove HTML artifacts & numbers
    - convert to lowercase then join as one string
    '''
    
    # Remove HTML artifacts using BeautifulSoup 
    body_text = BeautifulSoup(raw_body).get_text()
    
    # Remove non-letters (returns only letters) -- Regex 
    letters_only = re.sub("[^a-zA-Z]", " ", body_text)
    
    # Convert to lower case, split into individual words (tokenize)
    words = letters_only.lower().split()
    
    # Join the words back into one string separated by space, and return the result.
    return(" ".join(words))

In [23]:
# Execution time: ~5m 56s 
df['body_clean'] = df['Body'].map(lambda x: body_cleanup(x))

In [24]:
df.head()

,Id,CreationDate,Title,Body,Tags,ViewCount,AnswerCount,CommentCount,target_tags,overlap_tags,target_class,body_clean
1,53992219,2019-01-01 00:01:55,How to programmatically change style sheet of ...,<p>I have so many buttons on a dialog and I wa...,"[c++, qt, qt5, qtstylesheets, qpushbutton]",775,2,2,{c++},c++,4,i have so many buttons on a dialog and i want ...
4,53992223,2019-01-01 00:02:37,Unable to print a class list attribute using i...,<p>I am designing a deck class that has <stron...,"[python, python-3.x, list, class, printing]",40,2,0,"{python, python-3.x}",python python-3.x,5,i am designing a deck class that has init meth...
8,53992234,2019-01-01 00:05:48,How to rearrange subplots so that one is under...,<p>I am trying to code two plots such that one...,"[python, matplotlib, subplot]",519,1,1,{python},python,5,i am trying to code two plots such that one pl...
10,53992248,2019-01-01 00:09:24,Function always returns 1,<p>I´m trying to write a simple branch predict...,"[c++, function]",150,1,21,{c++},c++,4,i m trying to write a simple branch predictor ...
11,53992252,2019-01-01 00:11:20,possible to speed up this query?,<p>I have the following query which takes a li...,"[sql, postgresql]",40,1,0,{sql},sql,0,i have the following query which takes a littl...


## MISC Cleaning 

### Reset Index

In [25]:
df.reset_index(inplace = True)

In [28]:
df.head()

,index,Id,CreationDate,Title,Body,Tags,ViewCount,AnswerCount,CommentCount,target_tags,overlap_tags,target_class,body_clean
0,1,53992219,2019-01-01 00:01:55,How to programmatically change style sheet of ...,<p>I have so many buttons on a dialog and I wa...,"[c++, qt, qt5, qtstylesheets, qpushbutton]",775,2,2,{c++},c++,4,i have so many buttons on a dialog and i want ...
1,4,53992223,2019-01-01 00:02:37,Unable to print a class list attribute using i...,<p>I am designing a deck class that has <stron...,"[python, python-3.x, list, class, printing]",40,2,0,"{python, python-3.x}",python python-3.x,5,i am designing a deck class that has init meth...
2,8,53992234,2019-01-01 00:05:48,How to rearrange subplots so that one is under...,<p>I am trying to code two plots such that one...,"[python, matplotlib, subplot]",519,1,1,{python},python,5,i am trying to code two plots such that one pl...
3,10,53992248,2019-01-01 00:09:24,Function always returns 1,<p>I´m trying to write a simple branch predict...,"[c++, function]",150,1,21,{c++},c++,4,i m trying to write a simple branch predictor ...
4,11,53992252,2019-01-01 00:11:20,possible to speed up this query?,<p>I have the following query which takes a li...,"[sql, postgresql]",40,1,0,{sql},sql,0,i have the following query which takes a littl...


In [29]:
# Drop 'index' column (b/c I reset index above)
df = df.drop(['index'], axis = 1)

### Check for nulls

In [31]:
df.isnull().sum()

Id              0
CreationDate    0
Title           0
Body            0
Tags            0
ViewCount       0
AnswerCount     0
CommentCount    0
target_tags     0
overlap_tags    0
target_class    0
body_clean      0
dtype: int64

In [40]:
# In EDA notebook, find null, because body_clean is empty
# in above cleaning function, need to add: Nan if empty -- can fix for next iteration of project 
# df[df['Id'] == 55234780]

,Id,CreationDate,Title,Body,Tags,ViewCount,AnswerCount,CommentCount,target_tags,overlap_tags,target_class,body_clean
91170,55234780,2019-03-19 06:21:07,1st data is officeIn and 2nd data is OfficeOut...,"<p><a href=""https://i.stack.imgur.com/P3SNS.pn...","[mysql, sql]",26,1,0,"{mysql, sql}",mysql sql,0,


### Check Dtypes

In [34]:
df.dtypes

Id               int64
CreationDate    object
Title           object
Body            object
Tags            object
ViewCount        int64
AnswerCount      int64
CommentCount     int64
target_tags     object
overlap_tags    object
target_class     int64
body_clean      object
dtype: object

#### Datetime formatt 

In [35]:
df['CreationDate']= pd.to_datetime(df['CreationDate']) 

# Save as clean_final.csv

In [36]:
df.to_csv('../data/clean_final.csv', index=False)